In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import neurokit2 as nk
import matplotlib.pyplot as plt


from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

import tensorflow as tf
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Flatten, Conv1D,TimeDistributed, Reshape, Activation, add, Layer, Concatenate, GRU, Bidirectional 
from tensorflow.keras.layers import MaxPooling1D, BatchNormalization, Dropout,Activation, GlobalAveragePooling1D

In [ ]:
# R-peak detection
def get_rpeak(signal, sampling_rate):
    _, rpeaks = nk.ecg_peaks(signal, sampling_rate=sampling_rate, show=False, method='neurokit')
    
    return rpeaks['ECG_R_Peaks']

# Segmentation Function

def load_BEAT_data(signal, r_peaks):
    signal_list = []
    for number in range(1,len(r_peaks)-1):
        signal_list.append(signal[r_peaks[number]-(int(left_sec*sampling_rate)) : r_peaks[number]+(int(right_sec*sampling_rate))])
    return np.array(signal_list)

def load_RHYTHM_data(data, rpeak, window):
    rhythm_data = []
    right = int(right_sec * sampling_rate)
    for i in range(1,len(rpeak)-1):
        if len(data[rpeak[i] + right : rpeak[i] + window + right]) == window:
            rhythm_data.append(data[rpeak[i] + right : rpeak[i]+right + window])

    return np.array(rhythm_data)

def shuffle_data(data, labels):
    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    return data[indices], labels[indices]

def shuffle_data2(data1, data2, labels):
    # data1, data2, label을 입력받아 무작위로 섞고 (인덱스 위치는 동일하게) Return
    indices = np.arange(data1.shape[0])
    np.random.shuffle(indices)
    return data1[indices], data2[indices], labels[indices]

In [ ]:
def cal_class_weight(label):
    
    counts = np.bincount(label)
    
    amuse_num = counts[0]
    normal_num = counts[1]
    stress_num = counts[2]
    meditation_num = counts[3]
    total = amuse_num + normal_num + stress_num + meditation_num
    
    weight_amuse = (1/amuse_num) * (total/4.0)
    weight_normal = (1/normal_num) * (total/4.0)
    weight_stress = (1/stress_num) * (total/4.0)
    weight_meditation  = (1/meditation_num) * (total/4.0)
    class_weight = {0:weight_amuse, 1:weight_normal, 2:weight_stress, 3:weight_meditation}
    # print(total, class_weight)

    return class_weight

def Plot_lr_curve(history):
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy'] 
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    plt.figure(figsize=(20,5))
    plt.subplot(121)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend()
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')

    plt.subplot(122)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend()
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')

def Plot_confusion_matrix(true_d, pred, class_name):
    
    pred_label = []
    true_label = []
    
    for i in range(len(pred)):
        pred_label.append(np.argmax(pred[i]))
        
    for i in range(len(true_d)):
        true_label.append(np.argmax(true_d[i]))
    
    print(pred_label[0:10])
    # Accuracy
    accuracy = accuracy_score(true_label, pred_label)
    print('accuracy : ', np.round(accuracy,3))
    
    # Precision
    precision = precision_score(true_label, pred_label, average='macro')
    print('precision : ', np.round(precision,3))
    
    # Recall
    recall = recall_score(true_label, pred_label,average='macro')
    print('recall : ', np.round(recall,3))
    
    # F1 Score
    F1_score = f1_score(true_label, pred_label,average='macro')
    print('F1_score : ', np.round(F1_score,3))

    #clasification report
    report = classification_report(true_label, pred_label, target_names=class_name)#'Amuse',
    print(report)
    
    #confusion matrix
    confusion = confusion_matrix(true_label, pred_label)
    disp = ConfusionMatrixDisplay(confusion, display_labels=class_name)#'Amuse',
    disp.plot(cmap=plt.cm.Blues)
    plt.show()
    
    #plt.figure(figsize=(10,5))
    cmn = confusion.astype('float') / confusion.sum(axis=1)[:, np.newaxis]
    sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=class_name, yticklabels=class_name)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show(block=False)

In [ ]:
# import pickle5 as pickle

def load_data(db_path, segment, window=0):

    back_0_data, back_2_data, back_3_data, resting_data = [], [], [], []
    
    for i, p in enumerate(os.listdir(db_path)):
        
        # with open(db_path+p,"rb") as fr:
        #     df = pickle.load(fr, encoding='bytes')
            
        df = pd.read_pickle(db_path + p)
        # print(p)
        back_0 = np.concatenate((df['process Trial 1:0back'], df['process Trial 6:0back']))
        back_2 = np.concatenate((df['process Trial 2:2back'], df['process Trial 4:2back']))
        back_3 = np.concatenate((df['process Trial 3:3back'], df['process Trial 5:3back']))
        resting = df['Resting'].flatten()
    
        # print(back_0.shape, back_2.shape, back_3.shape, resting.shape)
    
        back_0_Rpeak = get_rpeak(back_0, sampling_rate)
        back_2_Rpeak = get_rpeak(back_2, sampling_rate)
        back_3_Rpeak = get_rpeak(back_3, sampling_rate)
        resting_Rpeak = get_rpeak(resting.flatten().flatten(), sampling_rate)
    
        if segment == 'beat':
            back_0_data.append(load_BEAT_data(back_0, back_0_Rpeak))
            back_2_data.append(load_BEAT_data(back_2, back_2_Rpeak))
            back_3_data.append(load_BEAT_data(back_3, back_3_Rpeak))
            resting_data.append(load_BEAT_data(resting, resting_Rpeak))
            
        else:
            back_0_data.append(load_RHYTHM_data(back_0, back_0_Rpeak, window))
            back_2_data.append(load_RHYTHM_data(back_2, back_2_Rpeak, window))
            back_3_data.append(load_RHYTHM_data(back_3, back_3_Rpeak, window))
            resting_data.append(load_RHYTHM_data(resting, resting_Rpeak, window))

    back_0_data = np.vstack(back_0_data)
    back_2_data = np.vstack(back_2_data)
    back_3_data = np.vstack(back_3_data)
    resting_data = np.vstack(resting_data)

    return back_0_data, back_2_data, back_3_data, resting_data


    return data, label

In [ ]:
left_sec = 0.24
right_sec = 0.4

sampling_rate=256

class_name = ['0_back', '2_back', '3_back', 'Resting']

In [ ]:
import os
db_path = "D:/Journal/MAUS_preprocessd_DB/"
person = os.listdir(db_path)

In [ ]:
window_list = [i*sampling_rate for i in range(2,11)]
print(window_list)

In [ ]:
# b_back_0, b_back_2, b_back_3, b_resting = load_data(db_path, 'beat')
r_back_0, r_back_2, r_back_3, r_resting = load_data(db_path, 'rhythm', sampling_rate*3)

In [ ]:
back_0_label = np.zeros(r_back_0.shape[0], np.int32)
back_2_label = np.ones(r_back_2.shape[0], np.int32) * 1
back_3_label = np.ones(r_back_3.shape[0], np.int32) * 2
resting_label = np.ones(r_resting.shape[0], np.int32) * 3

In [ ]:
# beat_data = np.concatenate([b_back_0, b_back_2, b_back_3, b_resting])
rhythm_data = np.concatenate([r_back_0, r_back_2, r_back_3, r_resting])
label = np.concatenate([back_0_label, back_2_label, back_3_label, resting_label])

In [ ]:
Beat, Beat_label = shuffle_data(rhythm_data, label)

In [ ]:
train_b, test_b, train_label_y, test_label_y = train_test_split(Beat, Beat_label,
                                                            test_size=0.2,
                                                            stratify=Beat_label,
                                                            random_state=128,
                                                            shuffle=True)

train_b, val_b, train_label_y, val_label_y = train_test_split(train_b, train_label_y,
                                                         test_size=0.25,
                                                         stratify=train_label_y,
                                                         random_state=128,
                                                         shuffle=True)

print(train_b.shape, val_b.shape, test_b.shape)

In [ ]:
train_label = to_categorical(train_label_y)
val_label = to_categorical(val_label_y)
test_label = to_categorical(test_label_y)

# Rhythm 5-Fold
## 5 layer, 3 seoncd
## CNN-2LSTM

In [ ]:
r_back_0, r_back_2, r_back_3, r_resting = load_data(db_path, 'rhythm', sampling_rate*3)

back_0_label = np.zeros(r_back_0.shape[0], np.int32)
back_2_label = np.ones(r_back_2.shape[0], np.int32) * 1
back_3_label = np.ones(r_back_3.shape[0], np.int32) * 2
resting_label = np.ones(r_resting.shape[0], np.int32) * 3

# beat_data = np.concatenate([b_back_0, b_back_2, b_back_3, b_resting])
rhythm_data = np.concatenate([r_back_0, r_back_2, r_back_3, r_resting])
label = np.concatenate([back_0_label, back_2_label, back_3_label, resting_label])

Rhythm, Rhythm_label = shuffle_data(rhythm_data, label)

In [ ]:
 # 가중치 초기화 
weight_init = HeNormal(seed= 128)

save_model_path = "D:/Journal/MAUS_Model_2/Rhythm_network/"

def scheuler(epoch, lr):
    if epoch < 41:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

callback_lr = tf.keras.callbacks.LearningRateScheduler(scheuler)

def cal_class_weight(label):
    
    counts = np.bincount(label)
    
    amuse_num = counts[0]
    normal_num = counts[1]
    stress_num = counts[2]
    meditation_num = counts[3]
    total = amuse_num + normal_num + stress_num + meditation_num
    
    weight_amuse = (1/amuse_num) * (total/4.0)
    weight_normal = (1/normal_num) * (total/4.0)
    weight_stress = (1/stress_num) * (total/4.0)
    weight_meditation  = (1/meditation_num) * (total/4.0)
    class_weight = {0:weight_amuse, 1:weight_normal, 2:weight_stress, 3:weight_meditation}
    # print(total, class_weight)

    return class_weight

In [ ]:
def rhythm_model(re_X_train_r):
    input_rhythm = Input(shape=(re_X_train_r,1), name = 'input_rhythm')

    conv1_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_rhythm)
    conv2_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R)
    bn1_R = BatchNormalization()(conv2_R)
    act_R = Activation('relu')(bn1_R)
    max1_R = MaxPooling1D(2,2)(act_R)

    conv1_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R)
    conv2_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_1)
    bn1_R_1 = BatchNormalization()(conv2_R_1)
    act_R_1 = Activation('relu')(bn1_R_1)
    max1_R_1 = MaxPooling1D(2,2)(act_R_1)


    conv1_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_1)
    conv2_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_2)
    bn1_R_2 = BatchNormalization()(conv2_R_2)
    act_R_2 = Activation('relu')(bn1_R_2)
    max1_R_2 = MaxPooling1D(2,2)(act_R_2)


    conv1_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_2)
    conv2_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_3)
    bn1_R_3 = BatchNormalization()(conv2_R_3)
    act_R_3 = Activation('relu')(bn1_R_3)
    max1_R_3 = MaxPooling1D(2,2)(act_R_3)


    conv1_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_3)
    conv2_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_4)
    bn1_R_4 = BatchNormalization()(conv2_R_4)
    act_R_4 = Activation('relu')(bn1_R_4)
    max1_R_4 = MaxPooling1D(2,2)(act_R_4)

    GAP_R = GlobalAveragePooling1D()(max1_R_4)

    re_R = Reshape((1, GAP_R.shape[1]))(GAP_R)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs= input_rhythm, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:
number = 1
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=128)

for train_index, test_index in skf.split(Rhythm, Rhythm_label):
    X_train_b, X_test_b = Rhythm[train_index], Rhythm[test_index]
    Y_train, Y_test = Rhythm_label[train_index], Rhythm_label[test_index]

    y_train = to_categorical(Y_train)
    y_test = to_categorical(Y_test)
    class_weight = cal_class_weight(Y_train)

    MAUS_beat_model = rhythm_model(X_train_b.shape[1])
    callback_lr = tf.keras.callbacks.LearningRateScheduler(scheuler)
    MAUS_beat_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = MAUS_beat_model.fit(X_train_b, y_train, 
                        validation_data=(X_test_b, y_test), 
                        epochs=50, 
                        batch_size=32,
                        callbacks=[callback_lr],
                        class_weight=class_weight
                       ) 
    
    
    scores = MAUS_beat_model.evaluate(X_test_b, y_test)
    Plot_lr_curve(history)
    pred = MAUS_beat_model.predict(X_test_b)
    Plot_confusion_matrix(y_test, pred, class_name)
    
    
    MAUS_beat_model.save(save_model_path + 'rhythm_8020_5_layer_(0.0001)_Fold_'  +str(number)+ '_(' + str(np.round(scores[1]*100,2)) + ').h5')
    number+=1


# Fusion Network

In [ ]:
b_back_0, b_back_2, b_back_3, b_resting = load_data(db_path, 'beat')
r_back_0, r_back_2, r_back_3, r_resting = load_data(db_path, 'rhythm', sampling_rate*3)

b_back_0 = b_back_0[:r_back_0.shape[0]]
b_back_2 = b_back_2[:r_back_2.shape[0]]
b_back_3 = b_back_3[:r_back_3.shape[0]]
b_resting = b_resting[:r_resting.shape[0]]

back_0_label = np.zeros(b_back_0.shape[0], np.int32)
back_2_label = np.ones(b_back_2.shape[0], np.int32) * 1
back_3_label = np.ones(b_back_3.shape[0], np.int32) * 2
resting_label = np.ones(b_resting.shape[0], np.int32) * 3

beat_data = np.concatenate([b_back_0, b_back_2, b_back_3, b_resting])
rhythm_data = np.concatenate([r_back_0, r_back_2, r_back_3, r_resting])
label = np.concatenate([back_0_label, back_2_label, back_3_label, resting_label])

Beat, Rhythm, Label = shuffle_data2(beat_data, rhythm_data, label)

In [ ]:
print(Beat.shape, Rhythm.shape, Label.shape)

In [ ]:
from tensorflow.keras.layers import Average, maximum, Multiply, Add, dot

def fusion_model_(re_X_train_b, re_X_train_r):
    input_beat = Input(shape=(re_X_train_b,1), name = 'input_beat')
    input_rhythm = Input(shape=(re_X_train_r,1), name = 'input_rhythm')

    conv1_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_beat)
    conv2_B = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B)
    bn1_B = BatchNormalization()(conv2_B)
    act_B = Activation('relu')(bn1_B)
    max1_B = MaxPooling1D(2,2)(act_B)

    conv1_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B)
    conv2_B_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_1)
    bn1_B_1 = BatchNormalization()(conv2_B_1)
    act_B_1 = Activation('relu')(bn1_B_1)
    max1_B_1 = MaxPooling1D(2,2)(act_B_1)


    conv1_B_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_1)
    conv2_B_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_2)
    bn1_B_2 = BatchNormalization()(conv2_B_2)
    act_B_2 = Activation('relu')(bn1_B_2)
    max1_B_2 = MaxPooling1D(2,2)(act_B_2)

    conv1_B_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_2)
    conv2_B_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_3)
    bn1_B_3 = BatchNormalization()(conv2_B_3)
    act_B_3 = Activation('relu')(bn1_B_3)
    max1_B_3 = MaxPooling1D(2,2)(act_B_3)

    conv1_B_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_B_3)
    conv2_B_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_B_4)
    bn1_B_4 = BatchNormalization()(conv2_B_4)
    act_B_4 = Activation('relu')(bn1_B_4)
    max1_B_4 = MaxPooling1D(2,2)(act_B_4)

    GAP_B = GlobalAveragePooling1D()(max1_B_4)

    # Rhythm

    conv1_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_rhythm)
    conv2_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R)
    bn1_R = BatchNormalization()(conv2_R)
    act_R = Activation('relu')(bn1_R)
    max1_R = MaxPooling1D(2,2)(act_R)

    conv1_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R)
    conv2_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_1)
    bn1_R_1 = BatchNormalization()(conv2_R_1)
    act_R_1 = Activation('relu')(bn1_R_1)
    max1_R_1 = MaxPooling1D(2,2)(act_R_1)


    conv1_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_1)
    conv2_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_2)
    bn1_R_2 = BatchNormalization()(conv2_R_2)
    act_R_2 = Activation('relu')(bn1_R_2)
    max1_R_2 = MaxPooling1D(2,2)(act_R_2)


    conv1_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_2)
    conv2_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_3)
    bn1_R_3 = BatchNormalization()(conv2_R_3)
    act_R_3 = Activation('relu')(bn1_R_3)
    max1_R_3 = MaxPooling1D(2,2)(act_R_3)


    conv1_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_3)
    conv2_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_4)
    bn1_R_4 = BatchNormalization()(conv2_R_4)
    act_R_4 = Activation('relu')(bn1_R_4)
    max1_R_4 = MaxPooling1D(2,2)(act_R_4)


    GAP_R = GlobalAveragePooling1D()(max1_R_4)

    #fusion
    # concatenated = Concatenate()([GAP_B, GAP_R])
    add_layer = maximum([GAP_B, GAP_R])

    re_R = Reshape((1, add_layer.shape[1]))(add_layer)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs=[input_beat, input_rhythm], outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=128)

save_model_path = "D:/Journal/MAUS_Model_2/Fusion_network/"
number = 1

for train_index, test_index in skf.split(Beat, Label):
    print('=========================================__Fold__'+str(number)+'_______=================================')
    X_train_b, X_test_b, X_train_r, X_test_r = Beat[train_index], Beat[test_index], Rhythm[train_index], Rhythm[test_index]
    Y_train, Y_test = Label[train_index], Label[test_index]

    y_train = to_categorical(Y_train)
    y_test = to_categorical(Y_test)
    class_weight = cal_class_weight(Y_train)
    
    MAUS_model = fusion_model_(X_train_b.shape[1], X_train_r.shape[1])
    callback_lr = tf.keras.callbacks.LearningRateScheduler(scheuler)
    MAUS_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = MAUS_model.fit([X_train_b, X_train_r], y_train, 
                        validation_data=([X_test_b, X_test_r], y_test), 
                        epochs=50,
                        batch_size=32,
                        callbacks=[callback_lr],
                        class_weight = class_weight
                       ) 

    
    scores = MAUS_model.evaluate([X_test_b,X_test_r], y_test)
    Plot_lr_curve(history)
    pred = MAUS_model.predict([X_test_b,X_test_r])
    Plot_confusion_matrix(y_test, pred, class_name)

    
    MAUS_model.save(save_model_path + 'MAUS_Fusion_Maximum_8020_Fold_'  +str(number)+ '_(' + str(np.round(scores[1]*100,2)) + ').h5')
    number+=1
    
    

# Beat layer setting experiment 
## layer (1-7), second (2-10) experiment

In [ ]:
from keras.callbacks import EarlyStopping
# 조기 종료 
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, min_delta=0.005, restore_best_weights=True)

# 가중치 초기화 
weight_init = HeNormal(seed= 128)

save_model_path = "D:/JH/Journal/MAUS_Model/Rhythm_layer_setting/"

#  layer 1

In [ ]:
def rhythm_model_1_layer(re_X_train_r):

    input_rhythm = Input(shape=(re_X_train_r,1), name = 'input_rhythm')

    # Rhythm

    conv1_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_rhythm)
    conv2_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R)
    bn1_R = BatchNormalization()(conv2_R)
    act_R = Activation('relu')(bn1_R)
    max1_R = MaxPooling1D(2,2)(act_R)

    GAP_R = GlobalAveragePooling1D()(max1_R)

    re_R = Reshape((1, GAP_R.shape[1]))(GAP_R)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs=input_rhythm, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model

In [ ]:
for window in window_list:
    print()
    print(f"===================={str(window)}=====================")
    print()
    r_back_0, r_back_2, r_back_3, r_resting = load_data(db_path, 'rhythm', window)

    back_0_label = np.zeros(r_back_0.shape[0], np.int32)
    back_2_label = np.ones(r_back_2.shape[0], np.int32) * 1
    back_3_label = np.ones(r_back_3.shape[0], np.int32) * 2
    resting_label = np.ones(r_resting.shape[0], np.int32) * 3

    rhythm_data = np.concatenate([r_back_0, r_back_2, r_back_3, r_resting])
    label = np.concatenate([back_0_label, back_2_label, back_3_label, resting_label])
    
    Beat, Beat_label = shuffle_data(rhythm_data, label)
    
    train_r, test_r, train_label_y, test_label_y = train_test_split(Beat, Beat_label,
                                                            test_size=0.2,
                                                            stratify=Beat_label,
                                                            random_state=128,
                                                            shuffle=True)

    train_r, val_r, train_label_y, val_label_y = train_test_split(train_r, train_label_y,
                                                             test_size=0.25,
                                                             stratify=train_label_y,
                                                             random_state=128,
                                                             shuffle=True)
    
    train_label = to_categorical(train_label_y)
    val_label = to_categorical(val_label_y)
    test_label = to_categorical(test_label_y)
    class_weight = cal_class_weight(train_label_y)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, min_delta=0.005, restore_best_weights=True)
    
    # model
    MAUS_rhythm_model = rhythm_model_1_layer(train_r.shape[1])
    MAUS_rhythm_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = MAUS_rhythm_model.fit(train_r, train_label, 
                              validation_data=(val_r, val_label), 
                              epochs=200, 
                              batch_size=32,
                              callbacks=[early_stopping],
                              class_weight=class_weight
                             )
    
    scores = MAUS_rhythm_model.evaluate(test_r, test_label)
    MAUS_rhythm_model.save(save_model_path + 'MAUS_Rhythm_(622)_layer(1)_(' + str(np.round(scores[1]*100,2)) +').h5' )
    Plot_lr_curve(history)
    pred = MAUS_rhythm_model.predict(test_r)
    Plot_confusion_matrix(test_label, pred, class_name)

# layer 2

In [ ]:
#  layer 2

def rhythm_model_2_layer(re_X_train_r):

    input_rhythm = Input(shape=(re_X_train_r,1), name = 'input_rhythm')

    # Rhythm

    conv1_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_rhythm)
    conv2_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R)
    bn1_R = BatchNormalization()(conv2_R)
    act_R = Activation('relu')(bn1_R)
    max1_R = MaxPooling1D(2,2)(act_R)
    
    conv1_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R)
    conv2_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_1)
    bn1_R_1 = BatchNormalization()(conv2_R_1)
    act_R_1 = Activation('relu')(bn1_R_1)
    max1_R_1 = MaxPooling1D(2,2)(act_R_1)

    GAP_R = GlobalAveragePooling1D()(max1_R_1)

    re_R = Reshape((1, GAP_R.shape[1]))(GAP_R)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs=input_rhythm, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model


In [ ]:

for window in window_list:
    print()
    print(f"===================={str(window)}=====================")
    print()
    r_back_0, r_back_2, r_back_3, r_resting = load_data(db_path, 'rhythm', window)

    back_0_label = np.zeros(r_back_0.shape[0], np.int32)
    back_2_label = np.ones(r_back_2.shape[0], np.int32) * 1
    back_3_label = np.ones(r_back_3.shape[0], np.int32) * 2
    resting_label = np.ones(r_resting.shape[0], np.int32) * 3

    rhythm_data = np.concatenate([r_back_0, r_back_2, r_back_3, r_resting])
    label = np.concatenate([back_0_label, back_2_label, back_3_label, resting_label])
    
    Beat, Beat_label = shuffle_data(rhythm_data, label)
    
    train_r, test_r, train_label_y, test_label_y = train_test_split(Beat, Beat_label,
                                                            test_size=0.2,
                                                            stratify=Beat_label,
                                                            random_state=128,
                                                            shuffle=True)

    train_r, val_r, train_label_y, val_label_y = train_test_split(train_r, train_label_y,
                                                             test_size=0.25,
                                                             stratify=train_label_y,
                                                             random_state=128,
                                                             shuffle=True)
    
    train_label = to_categorical(train_label_y)
    val_label = to_categorical(val_label_y)
    test_label = to_categorical(test_label_y)
    class_weight = cal_class_weight(train_label_y)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, min_delta=0.005, restore_best_weights=True)
    
    # model
    MAUS_rhythm_model = rhythm_model_2_layer(train_r.shape[1])
    MAUS_rhythm_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = MAUS_rhythm_model.fit(train_r, train_label, 
                              validation_data=(val_r, val_label), 
                              epochs=200, 
                              batch_size=32,
                              callbacks=[early_stopping],
                              class_weight=class_weight
                             )
    
    scores = MAUS_rhythm_model.evaluate(test_r, test_label)
    MAUS_rhythm_model.save(save_model_path + 'MAUS_Rhythm_(622)_'+str(window)+'_layer(2)_(' + str(np.round(scores[1]*100,2)) +').h5' )
    Plot_lr_curve(history)
    pred = MAUS_rhythm_model.predict(test_r)
    Plot_confusion_matrix(test_label, pred, class_name)

# layer 3

In [ ]:
#  layer 3

def rhythm_model_3_layer(re_X_train_r):

    input_rhythm = Input(shape=(re_X_train_r,1), name = 'input_rhythm')

    # Rhythm

    conv1_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_rhythm)
    conv2_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R)
    bn1_R = BatchNormalization()(conv2_R)
    act_R = Activation('relu')(bn1_R)
    max1_R = MaxPooling1D(2,2)(act_R)
    
    conv1_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R)
    conv2_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_1)
    bn1_R_1 = BatchNormalization()(conv2_R_1)
    act_R_1 = Activation('relu')(bn1_R_1)
    max1_R_1 = MaxPooling1D(2,2)(act_R_1)
    
    conv1_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_1)
    conv2_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_2)
    bn1_R_2 = BatchNormalization()(conv2_R_2)
    act_R_2 = Activation('relu')(bn1_R_2)
    max1_R_2 = MaxPooling1D(2,2)(act_R_2)

    GAP_R = GlobalAveragePooling1D()(max1_R_2)

    re_R = Reshape((1, GAP_R.shape[1]))(GAP_R)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs=input_rhythm, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model


In [ ]:
for window in window_list:
    print()
    print(f"===================={str(window)}=====================")
    print()
    r_back_0, r_back_2, r_back_3, r_resting = load_data(db_path, 'rhythm', window)

    back_0_label = np.zeros(r_back_0.shape[0], np.int32)
    back_2_label = np.ones(r_back_2.shape[0], np.int32) * 1
    back_3_label = np.ones(r_back_3.shape[0], np.int32) * 2
    resting_label = np.ones(r_resting.shape[0], np.int32) * 3

    rhythm_data = np.concatenate([r_back_0, r_back_2, r_back_3, r_resting])
    label = np.concatenate([back_0_label, back_2_label, back_3_label, resting_label])
    
    Beat, Beat_label = shuffle_data(rhythm_data, label)
    
    train_r, test_r, train_label_y, test_label_y = train_test_split(Beat, Beat_label,
                                                            test_size=0.2,
                                                            stratify=Beat_label,
                                                            random_state=128,
                                                            shuffle=True)

    train_r, val_r, train_label_y, val_label_y = train_test_split(train_r, train_label_y,
                                                             test_size=0.25,
                                                             stratify=train_label_y,
                                                             random_state=128,
                                                             shuffle=True)
    
    train_label = to_categorical(train_label_y)
    val_label = to_categorical(val_label_y)
    test_label = to_categorical(test_label_y)
    class_weight = cal_class_weight(train_label_y)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, min_delta=0.005, restore_best_weights=True)
    
    # model
    MAUS_rhythm_model = rhythm_model_3_layer(train_r.shape[1])
    MAUS_rhythm_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = MAUS_rhythm_model.fit(train_r, train_label, 
                              validation_data=(val_r, val_label), 
                              epochs=200, 
                              batch_size=32,
                              callbacks=[early_stopping],
                              class_weight=class_weight
                             )
    
    scores = MAUS_rhythm_model.evaluate(test_r, test_label)
    MAUS_rhythm_model.save(save_model_path + 'MAUS_Rhythm_(622)_'+str(window)+'_layer(3)_(' + str(np.round(scores[1]*100,2)) +').h5' )
    Plot_lr_curve(history)
    pred = MAUS_rhythm_model.predict(test_r)
    Plot_confusion_matrix(test_label, pred, class_name)

# layer 4

In [ ]:
#  layer 4

def rhythm_model_4_layer(re_X_train_r):

    input_rhythm = Input(shape=(re_X_train_r,1), name = 'input_rhythm')

    # Rhythm

    conv1_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_rhythm)
    conv2_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R)
    bn1_R = BatchNormalization()(conv2_R)
    act_R = Activation('relu')(bn1_R)
    max1_R = MaxPooling1D(2,2)(act_R)
    
    conv1_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R)
    conv2_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_1)
    bn1_R_1 = BatchNormalization()(conv2_R_1)
    act_R_1 = Activation('relu')(bn1_R_1)
    max1_R_1 = MaxPooling1D(2,2)(act_R_1)
    
    conv1_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_1)
    conv2_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_2)
    bn1_R_2 = BatchNormalization()(conv2_R_2)
    act_R_2 = Activation('relu')(bn1_R_2)
    max1_R_2 = MaxPooling1D(2,2)(act_R_2)
    
    conv1_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_2)
    conv2_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_3)
    bn1_R_3 = BatchNormalization()(conv2_R_3)
    act_R_3 = Activation('relu')(bn1_R_3)
    max1_R_3 = MaxPooling1D(2,2)(act_R_3)

    GAP_R = GlobalAveragePooling1D()(max1_R_3)

    re_R = Reshape((1, GAP_R.shape[1]))(GAP_R)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs=input_rhythm, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model


In [ ]:
for window in window_list:
    print()
    print(f"===================={str(window)}=====================")
    print()
    r_back_0, r_back_2, r_back_3, r_resting = load_data(db_path, 'rhythm', window)

    back_0_label = np.zeros(r_back_0.shape[0], np.int32)
    back_2_label = np.ones(r_back_2.shape[0], np.int32) * 1
    back_3_label = np.ones(r_back_3.shape[0], np.int32) * 2
    resting_label = np.ones(r_resting.shape[0], np.int32) * 3

    rhythm_data = np.concatenate([r_back_0, r_back_2, r_back_3, r_resting])
    label = np.concatenate([back_0_label, back_2_label, back_3_label, resting_label])
    
    Beat, Beat_label = shuffle_data(rhythm_data, label)
    
    train_r, test_r, train_label_y, test_label_y = train_test_split(Beat, Beat_label,
                                                            test_size=0.2,
                                                            stratify=Beat_label,
                                                            random_state=128,
                                                            shuffle=True)

    train_r, val_r, train_label_y, val_label_y = train_test_split(train_r, train_label_y,
                                                             test_size=0.25,
                                                             stratify=train_label_y,
                                                             random_state=128,
                                                             shuffle=True)
    
    train_label = to_categorical(train_label_y)
    val_label = to_categorical(val_label_y)
    test_label = to_categorical(test_label_y)
    class_weight = cal_class_weight(train_label_y)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, min_delta=0.005, restore_best_weights=True)
    
    # model
    MAUS_rhythm_model = rhythm_model_4_layer(train_r.shape[1])
    MAUS_rhythm_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = MAUS_rhythm_model.fit(train_r, train_label, 
                              validation_data=(val_r, val_label), 
                              epochs=200, 
                              batch_size=32,
                              callbacks=[early_stopping],
                              class_weight=class_weight
                             )
    
    scores = MAUS_rhythm_model.evaluate(test_r, test_label)
    MAUS_rhythm_model.save(save_model_path + 'MAUS_Rhythm_(622)_'+str(window)+'_layer(4)_(' + str(np.round(scores[1]*100,2)) +').h5' )
    Plot_lr_curve(history)
    pred = MAUS_rhythm_model.predict(test_r)
    Plot_confusion_matrix(test_label, pred, class_name)

# layer 5

In [ ]:
#  layer 5

def rhythm_model_5_layer(re_X_train_r):

    input_rhythm = Input(shape=(re_X_train_r,1), name = 'input_rhythm')

    # Rhythm

    conv1_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_rhythm)
    conv2_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R)
    bn1_R = BatchNormalization()(conv2_R)
    act_R = Activation('relu')(bn1_R)
    max1_R = MaxPooling1D(2,2)(act_R)
    
    conv1_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R)
    conv2_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_1)
    bn1_R_1 = BatchNormalization()(conv2_R_1)
    act_R_1 = Activation('relu')(bn1_R_1)
    max1_R_1 = MaxPooling1D(2,2)(act_R_1)
    
    conv1_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_1)
    conv2_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_2)
    bn1_R_2 = BatchNormalization()(conv2_R_2)
    act_R_2 = Activation('relu')(bn1_R_2)
    max1_R_2 = MaxPooling1D(2,2)(act_R_2)
    
    conv1_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_2)
    conv2_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_3)
    bn1_R_3 = BatchNormalization()(conv2_R_3)
    act_R_3 = Activation('relu')(bn1_R_3)
    max1_R_3 = MaxPooling1D(2,2)(act_R_3)
    
    conv1_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_3)
    conv2_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_4)
    bn1_R_4 = BatchNormalization()(conv2_R_4)
    act_R_4 = Activation('relu')(bn1_R_4)
    max1_R_4 = MaxPooling1D(2,2)(act_R_4)    

    GAP_R = GlobalAveragePooling1D()(max1_R_4)

    re_R = Reshape((1, GAP_R.shape[1]))(GAP_R)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs=input_rhythm, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model


In [ ]:
for window in window_list:
    print()
    print(f"===================={str(window)}=====================")
    print()
    r_back_0, r_back_2, r_back_3, r_resting = load_data(db_path, 'rhythm', window)

    back_0_label = np.zeros(r_back_0.shape[0], np.int32)
    back_2_label = np.ones(r_back_2.shape[0], np.int32) * 1
    back_3_label = np.ones(r_back_3.shape[0], np.int32) * 2
    resting_label = np.ones(r_resting.shape[0], np.int32) * 3

    rhythm_data = np.concatenate([r_back_0, r_back_2, r_back_3, r_resting])
    label = np.concatenate([back_0_label, back_2_label, back_3_label, resting_label])
    
    Beat, Beat_label = shuffle_data(rhythm_data, label)
    
    train_r, test_r, train_label_y, test_label_y = train_test_split(Beat, Beat_label,
                                                            test_size=0.2,
                                                            stratify=Beat_label,
                                                            random_state=128,
                                                            shuffle=True)

    train_r, val_r, train_label_y, val_label_y = train_test_split(train_r, train_label_y,
                                                             test_size=0.25,
                                                             stratify=train_label_y,
                                                             random_state=128,
                                                             shuffle=True)
    
    train_label = to_categorical(train_label_y)
    val_label = to_categorical(val_label_y)
    test_label = to_categorical(test_label_y)
    class_weight = cal_class_weight(train_label_y)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, min_delta=0.005, restore_best_weights=True)
    
    # model
    MAUS_rhythm_model = rhythm_model_5_layer(train_r.shape[1])
    MAUS_rhythm_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = MAUS_rhythm_model.fit(train_r, train_label, 
                              validation_data=(val_r, val_label), 
                              epochs=200, 
                              batch_size=32,
                              callbacks=[early_stopping],
                              class_weight=class_weight
                             )
    
    scores = MAUS_rhythm_model.evaluate(test_r, test_label)
    MAUS_rhythm_model.save(save_model_path + 'MAUS_Rhythm_(622)_'+str(window)+'_layer(5)_(' + str(np.round(scores[1]*100,2)) +').h5' )
    Plot_lr_curve(history)
    pred = MAUS_rhythm_model.predict(test_r)
    Plot_confusion_matrix(test_label, pred, class_name)

# layer 6

In [ ]:
#  layer 6

def rhythm_model_6_layer(re_X_train_r):

    input_rhythm = Input(shape=(re_X_train_r,1), name = 'input_rhythm')

    # Rhythm

    conv1_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_rhythm)
    conv2_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R)
    bn1_R = BatchNormalization()(conv2_R)
    act_R = Activation('relu')(bn1_R)
    max1_R = MaxPooling1D(2,2)(act_R)
    
    conv1_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R)
    conv2_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_1)
    bn1_R_1 = BatchNormalization()(conv2_R_1)
    act_R_1 = Activation('relu')(bn1_R_1)
    max1_R_1 = MaxPooling1D(2,2)(act_R_1)
    
    conv1_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_1)
    conv2_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_2)
    bn1_R_2 = BatchNormalization()(conv2_R_2)
    act_R_2 = Activation('relu')(bn1_R_2)
    max1_R_2 = MaxPooling1D(2,2)(act_R_2)
    
    conv1_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_2)
    conv2_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_3)
    bn1_R_3 = BatchNormalization()(conv2_R_3)
    act_R_3 = Activation('relu')(bn1_R_3)
    max1_R_3 = MaxPooling1D(2,2)(act_R_3)
    
    conv1_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_3)
    conv2_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_4)
    bn1_R_4 = BatchNormalization()(conv2_R_4)
    act_R_4 = Activation('relu')(bn1_R_4)
    max1_R_4 = MaxPooling1D(2,2)(act_R_4)    

    conv1_R_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_4)
    conv2_R_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_5)
    bn1_R_5 = BatchNormalization()(conv2_R_5)
    act_R_5 = Activation('relu')(bn1_R_5)
    max1_R_5 = MaxPooling1D(2,2)(act_R_5)     

    GAP_R = GlobalAveragePooling1D()(max1_R_5)

    re_R = Reshape((1, GAP_R.shape[1]))(GAP_R)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs=input_rhythm, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model


In [ ]:
for window in window_list:
    print()
    print(f"===================={str(window)}=====================")
    print()
    r_back_0, r_back_2, r_back_3, r_resting = load_data(db_path, 'rhythm', window)

    back_0_label = np.zeros(r_back_0.shape[0], np.int32)
    back_2_label = np.ones(r_back_2.shape[0], np.int32) * 1
    back_3_label = np.ones(r_back_3.shape[0], np.int32) * 2
    resting_label = np.ones(r_resting.shape[0], np.int32) * 3

    rhythm_data = np.concatenate([r_back_0, r_back_2, r_back_3, r_resting])
    label = np.concatenate([back_0_label, back_2_label, back_3_label, resting_label])
    
    Beat, Beat_label = shuffle_data(rhythm_data, label)
    
    train_r, test_r, train_label_y, test_label_y = train_test_split(Beat, Beat_label,
                                                            test_size=0.2,
                                                            stratify=Beat_label,
                                                            random_state=128,
                                                            shuffle=True)

    train_r, val_r, train_label_y, val_label_y = train_test_split(train_r, train_label_y,
                                                             test_size=0.25,
                                                             stratify=train_label_y,
                                                             random_state=128,
                                                             shuffle=True)
    
    train_label = to_categorical(train_label_y)
    val_label = to_categorical(val_label_y)
    test_label = to_categorical(test_label_y)
    class_weight = cal_class_weight(train_label_y)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, min_delta=0.005, restore_best_weights=True)
    
    # model
    MAUS_rhythm_model = rhythm_model_6_layer(train_r.shape[1])
    MAUS_rhythm_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = MAUS_rhythm_model.fit(train_r, train_label, 
                              validation_data=(val_r, val_label), 
                              epochs=200, 
                              batch_size=32,
                              callbacks=[early_stopping],
                              class_weight=class_weight
                             )
    
    scores = MAUS_rhythm_model.evaluate(test_r, test_label)
    MAUS_rhythm_model.save(save_model_path + 'MAUS_Rhythm_(622)_'+str(window)+'_layer(6)_(' + str(np.round(scores[1]*100,2)) +').h5' )
    Plot_lr_curve(history)
    pred = MAUS_rhythm_model.predict(test_r)
    Plot_confusion_matrix(test_label, pred, class_name)

# layer 7

In [ ]:
#  layer 7

def rhythm_model_7_layer(re_X_train_r):

    input_rhythm = Input(shape=(re_X_train_r,1), name = 'input_rhythm')

    # Rhythm

    conv1_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(input_rhythm)
    conv2_R = Conv1D(8, 5, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R)
    bn1_R = BatchNormalization()(conv2_R)
    act_R = Activation('relu')(bn1_R)
    max1_R = MaxPooling1D(2,2)(act_R)
    
    conv1_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R)
    conv2_R_1 = Conv1D(16, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_1)
    bn1_R_1 = BatchNormalization()(conv2_R_1)
    act_R_1 = Activation('relu')(bn1_R_1)
    max1_R_1 = MaxPooling1D(2,2)(act_R_1)
    
    conv1_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_1)
    conv2_R_2 = Conv1D(32, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_2)
    bn1_R_2 = BatchNormalization()(conv2_R_2)
    act_R_2 = Activation('relu')(bn1_R_2)
    max1_R_2 = MaxPooling1D(2,2)(act_R_2)
    
    conv1_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_2)
    conv2_R_3 = Conv1D(64, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_3)
    bn1_R_3 = BatchNormalization()(conv2_R_3)
    act_R_3 = Activation('relu')(bn1_R_3)
    max1_R_3 = MaxPooling1D(2,2)(act_R_3)
    
    conv1_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_3)
    conv2_R_4 = Conv1D(128, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_4)
    bn1_R_4 = BatchNormalization()(conv2_R_4)
    act_R_4 = Activation('relu')(bn1_R_4)
    max1_R_4 = MaxPooling1D(2,2)(act_R_4)    

    conv1_R_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_4)
    conv2_R_5 = Conv1D(256, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_5)
    bn1_R_5 = BatchNormalization()(conv2_R_5)
    act_R_5 = Activation('relu')(bn1_R_5)
    max1_R_5 = MaxPooling1D(2,2)(act_R_5)     
    
    conv1_R_6 = Conv1D(512, 3, activation=None, padding='same', kernel_initializer=weight_init)(max1_R_5)
    conv2_R_6 = Conv1D(512, 3, activation=None, padding='same', kernel_initializer=weight_init)(conv1_R_6)
    bn1_R_6 = BatchNormalization()(conv2_R_6)
    act_R_6 = Activation('relu')(bn1_R_6)
    max1_R_6 = MaxPooling1D(2,2)(act_R_6)      

    GAP_R = GlobalAveragePooling1D()(max1_R_6)

    re_R = Reshape((1, GAP_R.shape[1]))(GAP_R)
    x_R_LSTM = LSTM(128,return_sequences=True)(re_R)
    x_R_LSTM2 = LSTM(64, return_sequences=True)(x_R_LSTM)
    fl_f = Flatten()(x_R_LSTM2)

    den1 = Dense(128, activation='relu')(fl_f)
    den2 = Dense(64, activation='relu')(den1)

    fusion_output = Dense(4, activation='softmax')(den2)

    fusion_model = Model(inputs=input_rhythm, outputs= fusion_output)
    fusion_model.summary()
    
    return fusion_model


In [ ]:
for window in window_list:
    print()
    print(f"===================={str(window)}=====================")
    print()
    r_back_0, r_back_2, r_back_3, r_resting = load_data(db_path, 'rhythm', window)

    back_0_label = np.zeros(r_back_0.shape[0], np.int32)
    back_2_label = np.ones(r_back_2.shape[0], np.int32) * 1
    back_3_label = np.ones(r_back_3.shape[0], np.int32) * 2
    resting_label = np.ones(r_resting.shape[0], np.int32) * 3

    rhythm_data = np.concatenate([r_back_0, r_back_2, r_back_3, r_resting])
    label = np.concatenate([back_0_label, back_2_label, back_3_label, resting_label])
    
    Beat, Beat_label = shuffle_data(rhythm_data, label)
    
    train_r, test_r, train_label_y, test_label_y = train_test_split(Beat, Beat_label,
                                                            test_size=0.2,
                                                            stratify=Beat_label,
                                                            random_state=128,
                                                            shuffle=True)

    train_r, val_r, train_label_y, val_label_y = train_test_split(train_r, train_label_y,
                                                             test_size=0.25,
                                                             stratify=train_label_y,
                                                             random_state=128,
                                                             shuffle=True)
    
    train_label = to_categorical(train_label_y)
    val_label = to_categorical(val_label_y)
    test_label = to_categorical(test_label_y)
    class_weight = cal_class_weight(train_label_y)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, min_delta=0.005, restore_best_weights=True)
    
    # model
    MAUS_rhythm_model = rhythm_model_7_layer(train_r.shape[1])
    MAUS_rhythm_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = MAUS_rhythm_model.fit(train_r, train_label, 
                              validation_data=(val_r, val_label), 
                              epochs=200, 
                              batch_size=32,
                              callbacks=[early_stopping],
                              class_weight=class_weight
                             )
    
    scores = MAUS_rhythm_model.evaluate(test_r, test_label)
    MAUS_rhythm_model.save(save_model_path + 'MAUS_Rhythm_(622)_'+str(window)+'_layer(7)_(' + str(np.round(scores[1]*100,2)) +').h5' )
    Plot_lr_curve(history)
    pred = MAUS_rhythm_model.predict(test_r)
    Plot_confusion_matrix(test_label, pred, class_name)